In [10]:
import pandas as pd 
import numpy as np 
import yfinance as yf
from datetime import date
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression

In [38]:
# Tickers
MAG7_TICKERS = ['AAPL','MSFT','TSLA','NVDA','META','AMZN','GOOG']
MAG7_WEIGHTS = [0.0656,0.0643,0.0225,0.0699,0.0264,0.0426,0.0225]

INDEX_TICKER = ['SPY']
EXTRA_TICKERS = ['AES', 'LNT', 'AEE', 'AEP', 'AWK', 'ATO', 'CNP'] #, 'CMS', 'ED']

# Start/End Dates
start_date = date(2021,1,1)
end_date = date(2025,1,1)

# Yfinance Download
df_index = yf.download(tickers = INDEX_TICKER,start = start_date, end = end_date, auto_adjust = True)['Close']
df_mag7 = yf.download(tickers = MAG7_TICKERS,start = start_date, end = end_date, auto_adjust = True)['Close']
df_extra = yf.download(tickers = EXTRA_TICKERS,start = start_date, end = end_date, auto_adjust = True)['Close']

df_rf = yf.download(tickers= '^IRX',start = start_date,end = end_date)['Close']
df_rf = (df_rf / 100) / 252
# Log Returns
# Log Returns
df_index_ret = (df_index
             .pct_change()
             .dropna()
        )

df_mag7_ret = (df_mag7
            .pct_change()
            .dropna()
        )

df_extra_ret = (df_extra
            .pct_change()
            .dropna()
        )

df_mag7_ret.index = pd.to_datetime(df_mag7_ret.index).date

# Weighted MAG7 Returns (Estimate)
# df_mag7_ret['Seven_Weighted'] = np.sum(df_mag7_ret*MAG7_WEIGHTS, axis = 1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed


In [39]:
df_rf.index = pd.to_datetime(df_rf.index).date

In [40]:
for column in df_mag7_ret.columns:
    df_mag7_ret[column] = df_mag7_ret[column] - df_rf['^IRX']

for column in df_extra_ret.columns:
    df_extra_ret[column] = df_extra_ret[column] - df_rf['^IRX']

df_index['SPY'] = df_index['SPY'] - df_rf['^IRX']

In [41]:
def rolling_capm(asset_returns,index_returns,window_size):

    rolling_results = []
    for start_idx in range(len(index_returns) - window_size + 1):
        end_idx = start_idx + window_size
        window_data = asset_returns.iloc[start_idx:end_idx]
        date_range = window_data.index
        # Perform CAPM regressions for each asset in the window
        for ticker in asset_returns.columns:
            y = window_data[ticker].dropna()
            x = index_returns.loc[y.index].values.reshape(-1, 1)
            if len(window_data) < window_size:
                continue
            # Fit linear regression
            model = LinearRegression().fit(x, y)
            # y_pred = model.predict(x)
            beta = model.coef_[0]
            alpha = model.intercept_
            
            # Calculate R2 and MAE
            r2 = model.score(x, y)
            # mae = mean_absolute_error(y, y_pred)
            
            # Store results
            rolling_results.append({
                "Start_Date": date_range[0],
                "End_Date": date_range[-1],
                "Asset": ticker,
                'Window':window_size,
                "R2": r2,
                'Beta':beta,
                "Alpha":alpha
                # "MAE": mae,
            })
    return pd.DataFrame(rolling_results)

In [65]:
d = rolling_capm(df_extra_ret,df_index_ret,120)

In [66]:
d

,Start_Date,End_Date,Asset,Window,R2,Beta,Alpha
0,2021-01-05,2021-06-25,AEE,120,0.030732,0.243343,0.000459
1,2021-01-05,2021-06-25,AEP,120,0.051870,0.298840,0.000142
2,2021-01-05,2021-06-25,AES,120,0.284154,1.370036,-0.000444
3,2021-01-05,2021-06-25,ATO,120,0.003276,0.087411,0.000502
4,2021-01-05,2021-06-25,AWK,120,0.160478,0.668177,-0.000439
...,...,...,...,...,...,...,...
6190,2024-07-12,2024-12-31,AES,120,0.034427,0.520053,-0.003047
6191,2024-07-12,2024-12-31,ATO,120,0.061492,0.247651,0.001077
6192,2024-07-12,2024-12-31,AWK,120,0.007743,0.115437,-0.000832
6193,2024-07-12,2024-12-31,CNP,120,0.030179,0.276066,0.000452


In [67]:
rolling_df = d.pivot(index = 'End_Date',columns = 'Asset',values = 'R2')

In [68]:
fig = go.Figure()

for ticker in rolling_df.columns:

    fig.add_trace(
        go.Scatter(
            x = rolling_df.index,
            y = rolling_df[ticker],
            name = f'{ticker}'
        )
    )
fig.update_layout(title = 'CAPM Regression R2 (Utilities)')
fig.update_layout(
                  showlegend=True,
                  margin=dict(l=10, r=10, t=50, b=10),
                  legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),
                  width = 800,height = 550,
                  xaxis_title = 'Date',
                  xaxis=dict(title_standoff=3) 
                )
fig.update_yaxes(title = 'R2',range = [0,1])
fig.show()